In [1]:
import os
import pandas as pd
from nixtla.nixtla_client import NixtlaClient
from ipywidgets import Text, Password, Dropdown, IntText, DatePicker, Button, Output, HBox, VBox, Label
from IPython.display import display
USE_PLOTLY = False
try:
    import plotly.graph_objs as go
    USE_PLOTLY = True
except Exception:
    import matplotlib.pyplot as plt


In [2]:
file_path = Text(value=r'C:\Users\juesh\OneDrive\Documents\windsurf\stock_data\QQQ_stock_data.tsv', description='File')
api_key = Password(value='', description='API Key')
target_col = Dropdown(options=[], description='Target')
model_sel = Dropdown(options=['timegpt-1','timegpt-1-long-horizon'], value='timegpt-1', description='Model')
freq_sel = Dropdown(options=['B','D','W','MS','H'], value='B', description='Freq')
h_input = IntText(value=10, description='Horizon')
start_date = DatePicker(description='Start')
end_date = DatePicker(description='End')
run_btn = Button(description='Run')
reset_btn = Button(description='Reset')
msg_out = Output()
plot_out = Output()


In [3]:
def detect_columns(df):
    time_candidates = [c for c in df.columns if c.lower() in ('date','datetime','ds','timestamp')]
    time_col = time_candidates[0] if time_candidates else None
    num_cols = df.select_dtypes(include=['number']).columns.tolist()
    defaults = [c for c in ('Close','Adj Close','y') if c in df.columns]
    target_options = defaults if defaults else num_cols
    return time_col, target_options

def prepare_df(df, time_col, target):
    df = df.copy()
    df[time_col] = pd.to_datetime(df[time_col], errors='coerce')
    df = df.dropna(subset=[time_col, target])
    out = pd.DataFrame({'unique_id':'QQQ', 'ds':df[time_col], 'y':df[target]})
    out = out.sort_values('ds')
    return out

def compute_metrics(actual, pred):
    e = actual - pred
    mae = e.abs().mean()
    rmse = (e.pow(2).mean())**0.5
    mape = (e.abs() / actual.replace(0, pd.NA)).dropna().mean() * 100
    return mae, rmse, mape

def clean_series_for_freq(df, freq):
    df = df.sort_values('ds')
    df = df.drop_duplicates(subset=['ds'], keep='last')
    full_idx = pd.date_range(df['ds'].min(), df['ds'].max(), freq=freq)
    out = df.set_index('ds').reindex(full_idx)
    out.index.name = 'ds'
    out = out.reset_index()
    out['unique_id'] = out['unique_id'].fillna(method='ffill').fillna('QQQ')
    out['y'] = out['y'].fillna(method='ffill').fillna(method='bfill')
    return out[['unique_id','ds','y']]

def plot_series(train_df, f_df, actual_future):
    if USE_PLOTLY:
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=train_df['ds'], y=train_df['y'], name='Actual', mode='lines'))
        fig.add_trace(go.Scatter(x=f_df['ds'], y=f_df['TimeGPT'], name='Forecast', mode='lines'))
        if actual_future is not None:
            delta = actual_future['y'] - f_df.set_index('ds')['TimeGPT']
            fig.add_trace(go.Bar(x=delta.index, y=delta.values, name='Delta'))
        return fig
    plt.figure(figsize=(10,4))
    plt.plot(train_df['ds'], train_df['y'], label='Actual')
    plt.plot(f_df['ds'], f_df['TimeGPT'], label='Forecast')
    if actual_future is not None:
        delta = actual_future['y'] - f_df.set_index('ds')['TimeGPT']
        plt.bar(delta.index, delta.values, label='Delta')
    plt.legend()
    return None


In [4]:
def on_run_clicked(_):
    with msg_out:
        msg_out.clear_output()
    with plot_out:
        plot_out.clear_output()
    if not api_key.value:
        with msg_out:
            print('Missing API key')
        return
    os.environ['NIXTLA_API_KEY'] = api_key.value
    try:
        df_raw = pd.read_csv(file_path.value, sep='	')
    except Exception as e:
        with msg_out:
            print(f'Failed to load file: {e}')
        return
    time_col, targets = detect_columns(df_raw)
    if not time_col:
        with msg_out:
            print('No time column detected')
        return
    if not target_col.options:
        target_col.options = targets
        target_col.value = targets[0] if targets else None
    if not target_col.value:
        with msg_out:
            print('No target column selected')
        return
    df_full = prepare_df(df_raw, time_col, target_col.value)
    df_tgpt = df_full.copy()
    if start_date.value is not None:
        df_tgpt = df_tgpt[df_tgpt['ds'] >= pd.to_datetime(start_date.value)]
    if end_date.value is not None:
        df_tgpt = df_tgpt[df_tgpt['ds'] <= pd.to_datetime(end_date.value)]
    df_tgpt = clean_series_for_freq(df_tgpt, freq_sel.value)
    if df_tgpt.shape[0] < 5:
        with msg_out:
            print('Training window too small')
        return
    h = int(h_input.value)
    if h <= 0:
        with msg_out:
            print('Horizon must be positive')
        return
    client = NixtlaClient()
    try:
        f_df = client.forecast(df=df_tgpt[['unique_id','ds','y']], h=h, model=model_sel.value, freq=freq_sel.value)
    except Exception as e:
        with msg_out:
            print(f'Forecast error: {e}')
        return
    last_ds = df_tgpt['ds'].max()
    actual_future = None
    future_mask = f_df['ds'] > last_ds
    if future_mask.any():
        idx = f_df.loc[future_mask, 'ds']
        a = df_full.set_index('ds')['y'].reindex(idx)
        actual_future = pd.DataFrame({'y': a})
    with plot_out:
        fig = plot_series(df_tgpt, f_df, actual_future)
        if USE_PLOTLY:
            display(fig)
        else:
            display(plt.gcf())
    if actual_future is not None and actual_future['y'].notna().any():
        aligned = f_df.set_index('ds').join(actual_future, how='left')
        aligned = aligned.dropna(subset=['y'])
        mae, rmse, mape = compute_metrics(aligned['y'], aligned['TimeGPT'])
        with msg_out:
            print(f'MAE: {mae:.4f}')
            print(f'RMSE: {rmse:.4f}')
            print(f'MAPE: {mape:.2f}%')
            print(f'Points evaluated: {aligned.shape[0]} of {future_mask.sum()}')

def on_reset_clicked(_):
    target_col.options = []
    api_key.value = ''
    start_date.value = None
    end_date.value = None
    h_input.value = 10
    with msg_out:
        msg_out.clear_output()
    with plot_out:
        plot_out.clear_output()
run_btn.on_click(on_run_clicked)
reset_btn.on_click(on_reset_clicked)
ui = VBox([HBox([file_path, api_key, target_col, model_sel, freq_sel, h_input]), HBox([start_date, end_date, run_btn, reset_btn]), msg_out, plot_out])
display(ui)


In [5]:
preset_short_h = 10
preset_long_h = 60


In [6]:
h_input.value = preset_short_h
model_sel.value = 'timegpt-1'
on_run_clicked(None)
h_input.value = preset_long_h
model_sel.value = 'timegpt-1-long-horizon'
on_run_clicked(None)


INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Validating inputs...
